In [1]:
import feedparser
import numpy as np
import ssl

In [2]:
def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

In [3]:
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    p0Num = np.ones(numWords)
    p1Num = np.ones(numWords)
    p0Denom = 2.0
    p1Denom = 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num / p1Denom)
    p0Vect = np.log(p0Num / p0Denom)
    return p0Vect, p1Vect, pAbusive

In [4]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

In [5]:
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

In [6]:
def textParse(bigString):
    import re
    listOfTokens = re.split(r'\W+', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

In [7]:
def calcMostFreq(vocabList, fullText):
    import operator
    freqDict = {}
    for token in vocabList:
        freqDict[token] = fullText.count(token)
    sortedFreq = sorted(freqDict.items(), key=operator.itemgetter(1), reverse=True)
    return sortedFreq[:30]

In [8]:
def localWords(feed1, feed0):
    docList = []
    classList = []
    fullText = []
    minLen = min(len(feed1['entries']), len(feed0['entries']))
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)
    top30Words = calcMostFreq(vocabList, fullText)
    for pairW in top30Words:
        if pairW[0] in vocabList: vocabList.remove(pairW[0])
    trainingSet = range(2 * minLen)
    testSet = []
    for i in range(20):
        randIndex = int(np.random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del (list(trainingSet)[randIndex])
    trainMat = []
    trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V, p1V, pSpam = trainNB0(np.array(trainMat), np.array(trainClasses))
    errorCount = 0
    for docIndex in testSet:
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
    print('the error rate is: ', float(errorCount) / len(testSet))
    return vocabList, p0V, p1V

In [9]:
def getTopWords(ny, sf):
    vocabList, p0V, p1V = localWords(ny, sf)
    topNY = []
    topSF = []
    for i in range(len(p0V)):
        if p0V[i] > -6.0: topSF.append((vocabList[i], p0V[i]))
        if p1V[i] > -6.0: topNY.append((vocabList[i], p1V[i]))
    sortedSF = sorted(topSF, key=lambda pair: pair[1], reverse=True)
    print("SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**")
    for item in sortedSF:
        print(item[0])
    sortedNY = sorted(topNY, key=lambda pair: pair[1], reverse=True)
    print("NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**")
    for item in sortedNY:
        print(item[0])

In [10]:
if hasattr(ssl, '_create_unverified_context'):
    ssl._create_default_https_context = ssl._create_unverified_context
ny = feedparser.parse('http://www.nfcmag.com/rss/lastest.xml')
print(ny)
print(ny['entries'])
print(len(ny['entries']))
sf = feedparser.parse('http://www.nfcmag.com/rss/lastest.xml')
vocabList, pSF, pNY = localWords(ny, sf)
getTopWords(ny, sf)

{'feed': {'title': '最新文章', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'http://www.nfcmag.com/rss/lastest.xml', 'value': '最新文章'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'http://www.nfcmag.com/'}], 'link': 'http://www.nfcmag.com/', 'image': {'href': 'http://www.nfcmag.com/images/logo-banner.png', 'title': '南风窗', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'http://www.nfcmag.com/rss/lastest.xml', 'value': '南风窗'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'http://www.nfcmag.com/'}], 'link': 'http://www.nfcmag.com/'}, 'subtitle': '南风窗最新文章', 'subtitle_detail': {'type': 'text/html', 'language': None, 'base': 'http://www.nfcmag.com/rss/lastest.xml', 'value': '南风窗最新文章'}, 'language': 'zh-cn', 'tags': [{'term': '最新文章', 'scheme': None, 'label': None}], 'rights': '南风窗', 'rights_detail': {'type': 'text/plain', 'language': None, 'base': 'http://www.nfcmag.com/rss/lastest.xml', 'value': '南风窗'}}, 'entries': [{'title': '“地

the error rate is:  0.7
the error rate is:  0.4
SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**
提前进入竞选模式的特朗普
一些看似出格
或许是她的幸运
稳住房价和提振信心
其次看国家未来布局
已与欧盟达成自贸协定的日本如何回应美方类似的
主动发现并把握己方的优势
根据pua理论
走下神坛
习近平对特朗普说
这就是中国正在发生的事情
关于明星资本化
其作为陆海贸易通衢的潜力也将得到释放
中国愿意在多边国际体系中发挥更大作用
但如果众议院调查
这也是斯坦
对于陌生人她都怀着敌意
梅则留在唐宁街
将这种优势发挥到极致
更可能将延续颠覆美国传统外交的路线
在促进世界和平和繁荣方面共同肩负着重要责任
这叫作
原则的普及
都有重要影响
程序游击战
汗的治理下得到缓解
恐惧对方的套路
就有多超乎想象
毒丸条款
合作是双方最好的选择
是中国经济正在经历的三件事
每个环节
在安徽省缺乏其他领军城市的情况下
列举一大串明星的名字没有太大的意义
但不突破社会底线
经验告诉我们
最终导向
早已超过拼多多
还有四坑
以它为代表
他愿意相信自己有定力稳坐钓鱼船
这坑过后
敢想敢做
有很多案例
以及围绕它而生的城市群化
它的故事有多奇幻
这犹如一场外科手术
巴基斯坦被压抑的优势将逐渐显现
回望过去的20年
听证密集出现
40年的改革开放让中国社会焕发巨大活力
她有什么理由不抓住这次机会呢
将使其陷入
资本管制
任凭风浪起
他是不是pua的人
大城市化
成功的故事还将继续
一场打破即将到来的议会僵局的新公投
在多边国际舞台上
再向外辐射带动的发展模式值得关注
极大地改变了中国的国运
影响深远
掣肘其总体外交的开展
可能意味着英国留在欧洲
当你变得更加成熟和自信
安全局势更加稳定
全国近万个地方政府融资平台正面临着转型的巨大挑战
奠定了今天中东和东南欧地区国家
在日美今后的贸易谈判中
网络给予中国人的最大也最可悲的自由
是不把网络自由当作真实的自由
首先是看国家意志
墨锁定在中美贸易战中的美国阵营
五坑陷阱法
那么中国城市以及人群的分层
人生不也是如此么
尤其是亚文化中的青年亚文化
本能质疑
使一系列单一民族国家从俄奥土三大帝国的废墟中崛起
在那之后很长一段时间里
也会更